In [1]:
#Connecting to MongoDB server

import pymongo

myclient = pymongo.MongoClient("mongodb://localhost:27017/")

print(myclient.list_database_names())

['admin', 'config', 'local']


In [2]:
videos = myclient["videos"] #access videos collection
print(myclient.list_database_names())

['admin', 'config', 'local']


In [5]:
import pandas as pd
import numpy as np
import json

#Function to read json file as collection
def read_json(filename,encoding):
    with open(filename, 'r', encoding=encoding) as file:
        try:
            data = json.load(file)  # json.load() reads and decodes JSON from a file
            return data
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")

In [3]:
db = myclient["local"] #access local server
videos = db["videos"] #CREATE OPERATION: Create a collection

print(db.list_collection_names())

['mia1136_student2', 'startup_log', 'lab4', 'mia1136_student_details', '21mia1136_student_details', 'users', 'mia1136_student1', 'inventory', 'ads']


In [6]:
#UPDATE OPERATION: Insert document into collection
x = videos.insert_one(read_json("C:/Users/Admin/Documents/FALL 24-25 COURSES/CSE4077 - Recommender Systems/Recommender App/Scraped videos/Film & Animation/Inside Out 2 _ Official Trailer.json",
                               "utf-16"))

print(db.list_collection_names())

['mia1136_student2', 'startup_log', 'lab4', 'mia1136_student_details', 'videos', '21mia1136_student_details', 'users', 'mia1136_student1', 'inventory', 'ads']


In [7]:
videos

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'local'), 'videos')

In [9]:
#READ OPERATION: return all documents in a collection
for x in videos.find():
    print(x)

{'_id': ObjectId('6734b54770f1f59e8194c69d'), 'title': 'Inside Out 2 | Official Trailer', 'url': 'https://www.youtube.com/watch?v=LEjhY15eCx0', 'channel': {'url': 'https://www.youtube.com/@pixar', 'name': 'Pixar'}, 'tags': ['Pixar', 'Disney', 'Disney Pixar', 'Pixar Movie', 'Animation']}


In [15]:
#Inserting more video documents
addresses = [
    "C:/Users/Admin/Documents/FALL 24-25 COURSES/CSE4077 - Recommender Systems/Recommender App/Scraped videos/Education/L-3.0_ Divide and Conquer _ Algorithm.json",
    "C:/Users/Admin/Documents/FALL 24-25 COURSES/CSE4077 - Recommender Systems/Recommender App/Scraped videos/Science & Technology/Electrolysis of Water.json",
    "C:/Users/Admin/Documents/FALL 24-25 COURSES/CSE4077 - Recommender Systems/Recommender App/Scraped videos/Sports/NBA _UNEXPECTED Dunks_ MOMENTS.json",
    "C:/Users/Admin/Documents/FALL 24-25 COURSES/CSE4077 - Recommender Systems/Recommender App/Scraped videos/Gaming/Halo Infinite _ PC Overview.json",
    "C:/Users/Admin/Documents/FALL 24-25 COURSES/CSE4077 - Recommender Systems/Recommender App/Scraped videos/Music/Tina Turner - What's Love Got To Do With It (Official Music Video).json"
]

sample = []
for a in addresses:
    sample.append(read_json(a,"utf-16"))

In [17]:
#insert_many function
x = videos.insert_many(sample)

for x in videos.find():
    print(x)

{'_id': ObjectId('6734b54770f1f59e8194c69d'), 'title': 'Inside Out 2 | Official Trailer', 'url': 'https://www.youtube.com/watch?v=LEjhY15eCx0', 'channel': {'url': 'https://www.youtube.com/@pixar', 'name': 'Pixar'}, 'tags': ['Pixar', 'Disney', 'Disney Pixar', 'Pixar Movie', 'Animation']}
{'_id': ObjectId('6734ba1570f1f59e8194c69e'), 'title': 'L-3.0: Divide and Conquer | Algorithm', 'url': 'https://www.youtube.com/watch?v=I8w2XN0w-fQ', 'channel': {'url': 'https://www.youtube.com/@GateSmashers', 'name': 'Gate Smashers'}, 'tags': ['how divide and conquer algorithm works', 'how divide and conquer approach work', 'what is divide and conquer algorithm', 'why divide and conquer works', 'is divide and conquer dynamic programming', 'is divide and conquer binary search', 'divide and conquer in design and analysis of algorithm', 'divide and conquer technique', 'divide and conquer technique with example', 'algorithm', 'divide and conquer', 'recurrence relation', 'dsa interview questions', 'daa inte

In [18]:
#Return only video titles
for x in videos.find({},{"title":1}):
    print(x)

{'_id': ObjectId('6734b54770f1f59e8194c69d'), 'title': 'Inside Out 2 | Official Trailer'}
{'_id': ObjectId('6734ba1570f1f59e8194c69e'), 'title': 'L-3.0: Divide and Conquer | Algorithm'}
{'_id': ObjectId('6734ba1570f1f59e8194c69f'), 'title': 'Electrolysis of Water'}
{'_id': ObjectId('6734ba1570f1f59e8194c6a0'), 'title': 'NBA "UNEXPECTED Dunks" MOMENTS'}
{'_id': ObjectId('6734ba1570f1f59e8194c6a1'), 'title': 'Halo Infinite | PC Overview'}
{'_id': ObjectId('6734ba1570f1f59e8194c6a2'), 'title': "Tina Turner - What's Love Got To Do With It (Official Music Video)"}


In [24]:
#DELETE OPERATION: remove a video from the database
#videos.delete_one({'title': 'Electrolysis of Water'})
for x in videos.find({},{"_id":0,"title":1}):
    print(x)

{'title': 'Inside Out 2 | Official Trailer'}
{'title': 'L-3.0: Divide and Conquer | Algorithm'}
{'title': 'NBA "UNEXPECTED Dunks" MOMENTS'}
{'title': 'Halo Infinite | PC Overview'}
{'title': "Tina Turner - What's Love Got To Do With It (Official Music Video)"}


In [34]:
#Function to classify a video based on its tags

tag_db = read_json("tags_utf8.json","utf-8") #load tag database

def category(video_title):
    #Accessing metadata from title
    query = {"title":video_title}
    video = dict()
    for x in videos.find(query):
        video = x
    
    #Score dictionary to measure frequency of categories
    scores = {'Gaming':0,'Sports':0,'Education':0,'Autos & Vehicles':0,'Music':0,'Science & Technology':0,'Pets & Animals':0,
             'Food':0,'Film & Animation':0}
    
    #Split tags and remove duplicates
    split_tags = []
    for tag in video['tags']:
        split_tags.extend(tag.split(" ")) #Break down compound tags
        
    #Remove duplicates
    unique_tags = list(set(split_tags))
    
    #Iterate through each tag
    for tag in unique_tags:
        #Decapitalize tags
        if (tag[0]>='A')and(tag[0]<='Z'):
            tag = tag.lower()
        
        #Find the key of a matching tag
        for key, value in tag_db.items():
            if tag in value:
                #print(f"The key for the value '{tag}' is: {key}")
                scores[key] = scores[key] + 1
            
    cat = list(filter(lambda x: scores[x] == max(scores.values()), scores))[0]
    #print(f"This video belongs to {cat} category")
    #print(f"Scores = {scores.values()}")
    return cat

In [26]:
query = {"title":"Inside Out 2 | Official Trailer"}
test = dict()
for x in videos.find(query):
    #print(x)
    test = x

print(test)
print(test['title'])
print(test['tags'])
#classify = {"$set":{"category":category()}}

{'_id': ObjectId('6734b54770f1f59e8194c69d'), 'title': 'Inside Out 2 | Official Trailer', 'url': 'https://www.youtube.com/watch?v=LEjhY15eCx0', 'channel': {'url': 'https://www.youtube.com/@pixar', 'name': 'Pixar'}, 'tags': ['Pixar', 'Disney', 'Disney Pixar', 'Pixar Movie', 'Animation']}
Inside Out 2 | Official Trailer
['Pixar', 'Disney', 'Disney Pixar', 'Pixar Movie', 'Animation']


In [29]:
#Add category key to video document
sample_title = "Inside Out 2 | Official Trailer"
query = {"title":sample_title}
classify = {"$set":{"category":category(sample_title)}}

videos.update_one(query,classify)

for x in videos.find(query):
    print(x)

{'_id': ObjectId('6734b54770f1f59e8194c69d'), 'title': 'Inside Out 2 | Official Trailer', 'url': 'https://www.youtube.com/watch?v=LEjhY15eCx0', 'channel': {'url': 'https://www.youtube.com/@pixar', 'name': 'Pixar'}, 'tags': ['Pixar', 'Disney', 'Disney Pixar', 'Pixar Movie', 'Animation'], 'category': 'Film & Animation'}


In [36]:
#Classify multiple videos
for x in videos.find({}):
    videos.update_one({"title":x['title']},{"$set":{"category":category(x['title'])}})

for x in videos.find({},{"_id":0,"title":1,"category":1}):
    print(x)

{'title': 'Inside Out 2 | Official Trailer', 'category': 'Film & Animation'}
{'title': 'L-3.0: Divide and Conquer | Algorithm', 'category': 'Education'}
{'title': 'NBA "UNEXPECTED Dunks" MOMENTS', 'category': 'Sports'}
{'title': 'Halo Infinite | PC Overview', 'category': 'Gaming'}
{'title': "Tina Turner - What's Love Got To Do With It (Official Music Video)", 'category': 'Music'}
